<a href="https://colab.research.google.com/github/ishammansoor/AI-and-Machine-Learning/blob/main/Emotional_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
df = pd.read_csv("/content/text.csv")

Spliting into traning and testing sets

In [4]:
X=df['text']
y=df['label']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

Creating tokenizing the words using tensorflow and also padding it

In [6]:
tokenizer = Tokenizer(num_words = 393822, oov_token='')
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)

train_sequences = tokenizer.texts_to_sequences(X_train)
maxlen = max(len(tokens) for tokens in train_sequences)
test_sequences = tokenizer.texts_to_sequences(X_test)

print("Maximum sequence length (maxlen):", maxlen)

padded_training = pad_sequences(train_sequences, maxlen=maxlen, padding='post', truncating='post')
padded_testing = pad_sequences(test_sequences, maxlen=maxlen, padding='post', truncating='post')

Maximum sequence length (maxlen): 178


In [26]:
model = keras.Sequential([
   layers.Embedding()
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [27]:
predictions = model.fit(
)


Epoch 1/3
2206/2206 [==============================] - 70s 29ms/step - loss: 0.4741 - accuracy: 0.8315 - val_loss: nan - val_accuracy: 0.9173
Epoch 2/3
2206/2206 [==============================] - 35s 16ms/step - loss: 0.1564 - accuracy: 0.9315 - val_loss: nan - val_accuracy: 0.9087
Epoch 3/3
2206/2206 [==============================] - 34s 15ms/step - loss: 0.1148 - accuracy: 0.9423 - val_loss: nan - val_accuracy: 0.9275
